# Metadata

**L1 Taxonomy** - Problem Solving & Algorithms

**L2 Taxonomy** - Searching Algorithms

**Subtopic** - Linear Search Algorithm

**Use Case** - Implement a Linear Search algorithm in Python to find the first occurrence of a target value in a list of integers and return its index, with unit tests to validate its correctness.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
numpy==2.0.2
```


# Prompt

I run online store called Hashir's that handles custom packaging supplies for small businesses. We process thousands of orders daily and need a reliable utility to locate the first match of an order ID in a large batch quickly using a basic linear scan.

I'm wiring a quick Linear Search helper into my e-commerce fulfillment utilities and need it ready today.

```python
class OrderLookup:
    """
    Locate the first occurrence of a target order ID in a list of IDs
    using a plain linear search.
    """

    @staticmethod
    def find_order(orders: list[int], target: int) -> tuple[int, int]:
      pass
```

where orders is list of order ids to scan and target is order id to locate.

# Requirements

- Use pure linear search logic. For transparency and debugging, manually iterate through the list using basic `for` or `while` loops
- Stop scanning immediately after locating the first matching ID
- Do not modify the input list in any way
- Accept negative and zero IDs as valid values.
- Return `(-1, 0)` for an empty list without raising an exception
- Duplicate IDs must yield the earliest index.
- Worst case time complexity needs to be O(n) or better not worse
- Must reject lists containing any custom subclass of `int`. Also reject `bool` in validations
- Raise `TypeError` with these messages:
   - `"orders must be a list of integers"` if `orders` is not a `list`.
   - `"orders must contain only integers"` if any element of `orders` is not an `int`.
   - `"target must be an integer"` if `target` is not an `int`.
- Raise `ValueError` when the list length exceeds 10,000 with the exact message `"orders length exceeds 10,000"`
- Must return `(0, 1)` when the first element matches the target and return `(last_index, comparisons)` when only the final element matches
- Must return `(0, 1)` when all elements in the list are equal to the target.
- Make sure only one full traversal of `orders` happen in actual search
- Don't call `len(orders)` more than once. You may cache it in a local variable.
- Return a tuple `(index, comparisons)` where `index` is the position of the first match (-1 if not found) and `comparisons` is the number of equality checks performed during the search.
- Must handle `numpy.int32`, `numpy.int64` types as valid integers (not just built-in `int`)
- Must reject `decimal.Decimal` objects even if they represent whole numbers
- Must validate that no integer in the list exceeds `sys.maxsize` or is below `-sys.maxsize-1`
- Must validate that the target integer is within the range `[-sys.maxsize-1, sys.maxsize]`

In [4]:
# code

import sys
import numpy as np


class OrderLookup:
    """Locate first occurrence of target order ID by linear scan"""

    @staticmethod
    def find_order(orders: list[int], target: int) -> tuple[int, int]:
        if not isinstance(orders, list):
            raise TypeError("orders must be a list of integers")

        size = len(orders)
        if size > 10000:
            raise ValueError("orders length exceeds 10,000")

        allowed = (int, np.int32, np.int64)

        if type(target) not in allowed or isinstance(target, bool):
            raise TypeError("target must be an integer")
        if target < -sys.maxsize - 1 or target > sys.maxsize:
            raise ValueError("target must be within the valid integer range")

        if size == 0:
            return (-1, 0)

        for order in orders:
            if type(order) not in allowed or isinstance(order, bool):
                raise TypeError("orders must contain only integers")
            if order < -sys.maxsize - 1 or order > sys.maxsize:
                raise ValueError()

        comparisons = 0
        for idx in range(size):
            comparisons += 1
            if orders[idx] == target:
                return (idx, comparisons)

        return (-1, comparisons)

In [7]:
# tests

import unittest
import sys
import numpy as np
import decimal
from main import OrderLookup


class TestOrderLookup(unittest.TestCase):

    def test_empty_list_returns_minus1_and_zero_comparisons(self):
        self.assertEqual(OrderLookup.find_order([], 5), (-1, 0))

    def test_first_element_match(self):
        self.assertEqual(OrderLookup.find_order([10, 20, 30], 10), (0, 1))

    def test_last_element_match(self):
        self.assertEqual(OrderLookup.find_order([1, 2, 3, 4, 5], 5), (4, 5))

    def test_all_elements_match_target(self):
        self.assertEqual(OrderLookup.find_order([7, 7, 7], 7), (0, 1))

    def test_duplicate_ids_returns_earliest_index(self):
        self.assertEqual(OrderLookup.find_order([9, 3, 9, 9], 9), (0, 1))

    def test_target_not_found(self):
        self.assertEqual(OrderLookup.find_order([10, 20, 30], 99), (-1, 3))

    def test_type_error_non_list_input(self):
        with self.assertRaisesRegex(TypeError, "orders must be a list of integers"):
            OrderLookup.find_order("not a list", 1)

    def test_type_error_non_int_elements(self):
        with self.assertRaisesRegex(TypeError, "orders must contain only integers"):
            OrderLookup.find_order([1, 2.0, 3], 3)

    def test_type_error_for_bool_in_list(self):
        with self.assertRaisesRegex(TypeError, "orders must contain only integers"):
            OrderLookup.find_order([1, True, 3], 3)

    def test_type_error_for_target(self):
        with self.assertRaisesRegex(TypeError, "target must be an integer"):
            OrderLookup.find_order([1, 2, 3], "3")

    def test_value_error_for_list_exceeding_limit(self):
        with self.assertRaisesRegex(ValueError, "orders length exceeds 10,000"):
            OrderLookup.find_order([1] * 10001, 1)

    def test_valid_numpys_and_range_bounds(self):
      big = sys.maxsize
      small = -sys.maxsize - 1
      self.assertEqual(OrderLookup.find_order([np.int64(big), np.int64(small)], small), (1, 2))


    def test_decimal_in_orders_raises_type_error(self):
        with self.assertRaisesRegex(TypeError, "orders must contain only integers"):
            OrderLookup.find_order([1, decimal.Decimal(2), 3], 2)

    def test_large_integer_in_orders_raises_value_error(self):
        too_big = sys.maxsize + 1
        too_small = -sys.maxsize - 2
        with self.assertRaises(ValueError):
            OrderLookup.find_order([1, too_big], 1)
        with self.assertRaises(ValueError):
            OrderLookup.find_order([1, too_small], 1)

    def test_target_out_of_bounds_raises_value_error(self):
        too_big = sys.maxsize + 1
        too_small = -sys.maxsize - 2
        with self.assertRaises(ValueError):
            OrderLookup.find_order([1, 2, 3], too_big)
        with self.assertRaises(ValueError):
            OrderLookup.find_order([1, 2, 3], too_small)


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


ModuleNotFoundError: No module named 'main'

# Model Breaking Proof

#### Model Breaking Task URL: https://labeling-ms.turing.com/conversations/67676/view

#### Model code:

```python
# code generated by the model

# <Issue>: The model code does not raise exception if custom subclass of int is passed in orders list which violates requirement
# <Issue>: there is mismatch in function signature. The prompt used python built-in list, tuple while model used List, Tuple from typing module.
# <Issue>: custom subclass of int is allowed in target which violates explicit requirement

import sys
import numpy as np
from typing import List, Tuple

class OrderLookup:
    """
    Locate the first occurrence of a target order ID in a list of IDs
    using a plain linear search.
    """

    @staticmethod
    def find_order(orders: List[int], target: int) -> Tuple[int, int]:
        # Validate that orders is a list
        if not isinstance(orders, list):
            raise TypeError("orders must be a list of integers")

        # Check the length of the orders list
        length = len(orders)
        if length > 10000:
            raise ValueError("orders length exceeds 10,000")
        
        # Validate elements in the orders list
        for idx, order in enumerate(orders):
            if not isinstance(order, (int, np.int32, np.int64)) or isinstance(order, bool):
                raise TypeError("orders must contain only integers")
            if not (-sys.maxsize - 1 <= order <= sys.maxsize):
                raise ValueError("orders must be within the valid integer range")
        
        # Validate the target
        if not isinstance(target, (int, np.int32, np.int64)) or isinstance(target, bool):
            raise TypeError("target must be an integer")
        if not (-sys.maxsize - 1 <= target <= sys.maxsize):
            raise ValueError("target must be within the valid integer range")

        # Initialize variables for linear search
        comparisons = 0

        # Perform linear search
        for index in range(length):
            comparisons += 1
            if orders[index] == target:
                return (index, comparisons)

        # If no match is found, return (-1, number of comparisons)
        return (-1, comparisons)

# Usage Example:
# orders_list = [10, 23, 3, 4, 15]
# target_order_id = 15
# result = OrderLookup.find_order(orders_list, target_order_id)
# print(result)

# code generated by the model
```